In [1]:
from bokeh.plotting import output_notebook, figure, show
output_notebook()

import caret_analyze as caret
import caret_analyze.plot as caret_plot

Loading BokehJS ...

Succeed to find record_cpp_impl. the C++ version will be used.


In [2]:
# トレース結果の読み込み、 キャッシュを使わない場合は force_conversion=True とする。
lttng = caret.Lttng('end_to_end_sample', force_conversion=True)
# アーキテクチャファイルを読み込み、トレース結果を紐付ける
app = caret.Application('architecture.yaml', 'yaml', lttng)
# 評価対象のパス名を指定。
path = app.path['target_path']

found converted file but will re-create it: end_to_end_sample/converted
converting trace directory: end_to_end_sample
converted 6349 events in 874 ms
output written to: end_to_end_sample/converted
 [99%] [Ros2Handler]


In [3]:
# メッセージフローの出力
# granularity は raw/callback/node のいずれか
caret_plot.message_flow(path, granularity='node', treat_drop_as_delay=True)

In [ ]:
caret_plot.chain_latency(path, granularity='node', treat_drop_as_delay=True)

In [ ]:
t, latency_ns = path.to_timeseries(remove_dropped=False, treat_drop_as_delay=True)
latency_ms = latency_ns * 1.0e-6

p = figure()
p.line(t, latency_ns)
show(p)

In [ ]:
bins, hist = path.to_histogram(treat_drop_as_delay=True)
p = figure()
p.step(hist[1:], bins)
show(p)

In [ ]:
callback = app.callbacks[0] # コールバック実行時間
# node = app.nodes[0].paths[0] # ノードレイテンシ
path = app.paths[0] # パスレイテンシ
comm = app.communications[0] # 通信レイテンシ（pub_sub レイテンシ）
# comm_dds = app.communications[0].to_dds_latency() # 通信レイテンシ（DDSレイヤーレイテンシ） fastdds のみ。
var_pass = app.variable_passings[0] # 変数を介したメッセージ渡し